In [2]:
from sklearn.neighbors import KDTree
from collections import Counter
import numpy as np

import random

from itertools import combinations

import itertools

In [3]:
def nearNeighborsSet(data, labels, k, input_x, y=None, n=0):
    tree = KDTree(data)  # Construit un arbre KD à partir des points
    dists, indices = tree.query([input_x], k=k+n)  # Trouve les k+n voisins les plus proches

    neighbors = [(data[i], labels[i]) for i in indices[0]]  # Récupère les voisins avant suppression

    if y is not None and n > 0:
       # Keep all neighbors except those with label y
        neighbors_without_y = [neighbor for neighbor in neighbors if neighbor[1] != y]
        # If there are not enough neighbors without label y to fill k spots,
        # take as many as possible, then fill up with neighbors with label y
        if len(neighbors_without_y) < k:
            neighbors_with_y = [neighbor for neighbor in neighbors if neighbor[1] == y][:k - len(neighbors_without_y)]
            neighbors = neighbors_without_y + neighbors_with_y
        else:
            # If there are enough neighbors without label y, take the top k
            neighbors = neighbors_without_y[:k]
    else:
        # If y is not specified, just return the k+n neighbors
        neighbors = neighbors[:k+n]

    return neighbors

In [4]:
def mostFreqLabel(data, labels, k, input_x, y=None, n=0):
    neighbors = nearNeighborsSet(data, labels, k, input_x, y, n)
    neighbor_labels = [label for _, label in neighbors]
    return Counter(neighbor_labels).most_common(1)[0][0]

In [5]:
def QuickCertify(data, labels, n, input_x, K):
    label_set = []
    Kset = K  # Ensemble de valeurs de K à tester

    for k in Kset:
        # Obtenir l'étiquette la plus fréquente sans suppression
        y_initial = mostFreqLabel(data, labels, k, input_x)
        label_set.append(y_initial)
        # Obtenir l'étiquette la plus fréquente avec suppression de n éléments de label y
        y_after_removal = mostFreqLabel(data, labels, k, input_x, y_initial, n)

        if y_initial != y_after_removal:
            return False

        if len(label_set) != 1:
            return False

    return True

In [6]:
# # Charger l'ensemble de données Iris
# from sklearn.datasets import load_iris


# X, y = load_iris(return_X_y=True)

# # Exemple d'utilisation avec l'ensemble de données Iris
# input_x = X[50]  # Utiliser un élément de l'ensemble Iris comme exemple
# n = 5  # Nombre d'éléments à considérer comme potentiellement empoisonnés
# label_y = y[50]  # Utiliser le label réel de l'input_x comme exemple de label à supprimer

# # Appliquer QuickCertifyKD sur l'ensemble de données Iris
# result = QuickCertify(X, y, n, input_x, label_y)

KNN LEARN INIT

In [7]:
def partition(T, p):
    random.shuffle(T)
    return [T[i::p] for i in range(p)]

In [8]:
def KNNPredict(data, x, k):

    # Convertir l'ensemble d'apprentissage en un tableau numpy pour les caractéristiques et les étiquettes
    X_train = np.array([x for x, _ in data])
    y_train = np.array([y for _, y in data])

    # Créer un KDTree à partir de l'ensemble d'apprentissage
    tree = KDTree(X_train)

    # Trouver les indices des k voisins les plus proches
    dists, indices = tree.query([x], k=k)

    # Obtenir les étiquettes des k voisins les plus proches
    k_nearest_labels = y_train[indices[0]]

    # Retourner l'étiquette la plus commune parmi les k voisins
    return Counter(k_nearest_labels).most_common(1)[0][0]

In [9]:

def KNNLearnInit(T, p, K):
    # Partitionner l'ensemble d'apprentissage T en p groupes (G1, G2, ..., Gp)
    groups = partition(T, p)

    candidate_values = K

    errors = {Ki: [0 for _ in range(p)] for Ki in candidate_values}

    errSet = [[[] for _ in range(p)] for _ in candidate_values]

    for Ki in candidate_values:
        # Calculer i la position de Ki dans candidate_values
        i = candidate_values.index(Ki)

        for j, Gj in enumerate(groups):
            for x, y in Gj:
                data = [x for x in T if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in Gj)]
                if KNNPredict(data, x, Ki) != y:
                    errSet[i][j].append((x,y))
            errors[Ki][j] = len(errSet[i][j]) / len(Gj)


        errors[Ki] = sum([errors[Ki][j] for j in range(p)]) / p

    # Obtenir Ki avec l'erreur minimale
    K_star = min(errors, key=errors.get)

    Error_star = [(groups[j], errSet[candidate_values.index(K_star)][j]) for j in range(p)]

    # for j in range(p):
    #     print("G" + str(j+1))
    #     print("Ki: ", K_star)
    #     print("Error: ", errors[K_star])
    #     print("Gj: ", groups[j])
    #     print("ErrSet: ", errSet[candidate_values.index(K_star)][j])


    return K_star, Error_star



In [10]:
# X, y = load_iris(return_X_y=True)

# # # Créer une liste de paires (x, y) à partir de X et y
# xy = list(zip(X, y))

# k, error = KNNLearnInit(xy, 10)
# print(k)
# print(error[0][1])
# print(error[1][1])

GEN PROMISING SUBSETS

In [11]:
def generateSubsetsR1(data, k, input_x, n, min_rmv):
    """
    Génère des sous-ensembles R1 à partir des k+n éléments les plus proches de x.
    """
    features = [point[0] for point in data]
    labels = [point[1] for point in data]
    nearest_neighbors = nearNeighborsSet(features, labels, k, input_x, n=n)

    subsets_r1 = [list(subset) for r in range(min_rmv, len(nearest_neighbors) + 1)
                                for subset in combinations(nearest_neighbors, r)]
    return subsets_r1



In [12]:
def generateSubsetsR2(data, k, input_x, n, len_r1):
    """
    Supprime d'abord tous les k+n éléments les plus proches de x dans data, puis crée tous les sous-ensembles
    d'au plus n-len(r1) éléments avec le nouvel ensemble de données.
    """
    features = [point[0] for point in data]
    labels = [point[1] for point in data]
    nearest_neighbors = nearNeighborsSet(features, labels, k, input_x, n=n)

    # Utiliser la conversion en tuple pour la comparaison et éviter l'erreur
    reduced_data =  [x for x in data if not any(np.array_equal(x, element) for element in nearest_neighbors)]

    max_subset_size = n - len_r1
    subsets_r2 = []
    for r in range(1, max_subset_size + 1):
        for subset in combinations(reduced_data, r):
            subsets_r2.append(subset)
    return subsets_r2


In [13]:
def GenPromisingSubset(data, input_x, n, y, K):
    """
    Utilise une recherche binaire pour trouver min_rmv, puis génère des sous-ensembles R1 et R2.
    """
    promising_subsets = []
    Kset = K  # Ensemble des valeurs de K à considérer

    features = [point[0] for point in data]
    labels = [point[1] for point in data]

    for k in Kset:

        # Initialiser les variables pour la recherche binaire
        start, end = 1, n  # On commence à 1 car 0 ne changerait pas l'ensemble des voisins

        # Recherche binaire pour trouver le min_rmv optimal
        while start <= end:
            mid = (start + end) // 2
            if y == mostFreqLabel(features, labels, k, input_x, y, mid):
                start = mid + 1
            else:
                end = mid - 1

        min_rmv = start  # Le nombre minimal de suppressions pour changer la prédiction

        # Générer des sous-ensembles R1 avec au moins min_rmv éléments
        subsets_r1 = generateSubsetsR1(data, k, input_x, n, min_rmv)

        for r1 in subsets_r1:
            # Générer des sous-ensembles R2 avec le reste des données, jusqu'à n-len(r1) éléments
            subsets_r2 = generateSubsetsR2(data, k, input_x, n, len(r1))

            for r2 in subsets_r2:
                # Combinaison de R1 et R2 pour former un sous-ensemble prometteur
                R = r1 + list(r2)
                # Retirer les éléments de R de l'ensemble de données original pour obtenir le sous-ensemble final
                T_without_R = [x for x in data if not any(np.array_equal(x, element) for element in R)]

                promising_subsets.append(T_without_R)

    return promising_subsets


In [14]:
# from sklearn.datasets import load_iris
# X, y = load_iris(return_X_y=True)

# # Créer une liste de paires (x, y) à partir de X et y

# xy = list(zip(X, y))
# # print((xy))

# # Afficher les 5 premières paires
# print(len(GenPromisingSubset(data=xy[:149], input_x=[4.9, 3. , 1.4, 0.2], n=3, y='0', K=[3, 5, 7])))


KNN LEARN UPDATE

In [15]:
def compute_inflSet(R, groups):
    """
    Calcule l'ensemble des points influencés par R.

    :param R: Ensemble des points à supprimer.
    :param groups: Les groupes actuels de points de données.
    :return: Un ensemble de points influencés par R.
    """
    # Pour simplifier, disons que tous les points sont influencés par R.
    # Dans une implémentation réelle, vous devrez vérifier si la suppression
    # de points dans R affecte les voisins les plus proches des points dans groups.

    result = [x for x in groups if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in R)]
    return result


In [16]:
def ensemble(errSetGj_Ki, R, newSet_minus, newSet_plus):

    # Retirer les indices R et newSet_minus de errSetGj_Ki
    result = np.setdiff1d(errSetGj_Ki, R)
    result = np.setdiff1d(errSetGj_Ki, newSet_minus)

    # Ajouter les indices de newSet_plus à errSetGj_Ki
    result = np.union1d(errSetGj_Ki, newSet_plus)

    # Le tableau errSetGj_Ki est maintenant mis à jour
    return result


In [17]:
def KNNLearnUpdate(T, R, Error, candidate_k_values):
    """
    Mise à jour incrémentielle du modèle KNN.

    :param R: Ensemble des éléments à supprimer de l'entraînement.
    :param Error: Contient des groupes et des ensembles d'erreurs.
    :param candidate_k_values: Les valeurs k candidates pour le KNN.
    :return: Le meilleur k basé sur la plus petite erreur.
    """
    # Récupération des groupes et des ensembles d'erreurs
    groups = []
    err_sets = []
    for elt in Error:
        groups.append(elt[0])
        err_sets.append(elt[1])

    # Calcul des nouveaux groupes en supprimant R
    new_groups = []
    for elt in groups:
        ensemble = [x for x in elt if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in R)]
        new_groups.append(ensemble)

    # Création du nouvel ensemble d'entraînement T'
    T_prime = []
    for elt in new_groups:
        for i in elt:
            T_prime.append(i)

    # Initialisation de l'ensemble influencé par R
    inflSet = compute_inflSet(R, groups)  # Cette fonction doit être définie

    # Initialisation du dictionnaire pour les erreurs de chaque valeur k
    error_prime = []

    err_sets_prime = []

    errors_for_k = []

    # Itération sur chaque valeur k candidate
    for k in candidate_k_values:
        i = candidate_k_values.index(k)
        # Itération sur chaque groupe
        for j, Gj_prime in enumerate(new_groups):
            newSet_minus = []
            newSet_plus = []
            # Itération sur chaque élément de données dans le groupe influencé
            interGj_prime_inflSet = [x for x in Gj_prime if any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in inflSet)]
            for x, y in interGj_prime_inflSet:
                # Prédiction sur l'ensemble d'entraînement original
                data = [x for x in T if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in groups[j])]
                data_prime = [x for x in T_prime if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in Gj_prime)]
                if KNNPredict(data, x, k) == y and KNNPredict(data_prime, x, k) != y:
                    newSet_plus.add((x, y))

                if KNNPredict(data, x, k) != y and KNNPredict(data_prime, x, k) == y:
                    newSet_minus.add((x, y))

            # Mise à jour de l'ensemble des erreurs pour le groupe j
            err_sets_prime[i][j] = ensemble(err_sets[j], R, newSet_minus, newSet_plus)
            error_prime[i][j] = len(err_sets[Gj_prime]) / len(Gj_prime)

        # Calcul de l'erreur moyenne pour la valeur k
        avg_error_k = sum([error_prime[i][j] for j in range(len(groups))]) / len(groups)
        errors_for_k[i] = avg_error_k

    # Trouver le meilleur k avec la plus petite erreur moyenne
    best_k = min(errors_for_k, key=errors_for_k.get)

    return best_k

MAIN

In [18]:
def main(data, n, x):

    X = [point[0] for point in data]
    y = [point[1] for point in data]


    Kcandidates = [1, 2 , 4, 6, 8]

    if QuickCertify(X, y, n, x, Kcandidates):
        return 'Certified'

    K, Error = KNNLearnInit(data, 10, Kcandidates)

    y = KNNPredict(data, x, K)

    promised_set = GenPromisingSubset(data, x, n, y, Kcandidates)

    while len(promised_set) > 0:
        set_prime = promised_set[0]
        promised_set.pop(0)

        R = [x for x in data if not any([np.array_equal(x[0], y[0]) for y in set_prime])]
        K_prime = KNNLearnUpdate(data, R, Error, Kcandidates)
        y_prime = KNNPredict(set_prime, x, K_prime)

        if y != y_prime:
            return 'Falsified', R

    if len(promised_set) == 0:
        return 'Certified'
    else:
        return 'Unknown'

TESTTT

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)

# Create a list of pairs (x, y) from X and y
data = list(zip(X, y))

# Initialize counters
falsification = 0
certificat = 0
inconnu = 0

# Set the poisoning threshold
seuil_poisoning = 3

# Select 15 random data points
i = 0
data_input = []
while i < 15:
    index = np.random.randint(0, len(data))
    data_input.append(data[index])

    # Remove the selected data point from the list
    data.pop(index)

    i += 1

# Classify each data point
for element in data_input:
    print(element)
    result = main(data, seuil_poisoning, element[0])
    if result == 'Falsified':
        falsification += 1
    elif result == 'Certified':
        certificat += 1
    else:
        inconnu += 1

# Print the results
print('Falsified:', falsification)
print('Certified:', certificat)
print('Unknown:', inconnu)